In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import warnings
import time
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

url = 'https://www.thecocktailproject.com/search-recipes/'
driver.get(url) # go to website
time.sleep(1)

month_input = driver.find_element(By.ID, 'edit-month-of-birth')
month_input.send_keys('12')

day_input = driver.find_element(By.ID, 'edit-day-of-birth')
day_input.send_keys('07')

year_input = driver.find_element(By.ID, 'year')
year_input.send_keys('1999')
time.sleep(1)

app_submit = driver.find_element(By.CSS_SELECTOR, 'app-submit')
button_within_submit = app_submit.find_element(By.CSS_SELECTOR, 'button#edit-submit')
button_within_submit.click()
time.sleep(1)

reject_all_button = driver.find_element(By.ID, 'onetrust-reject-all-handler')
reject_all_button.click()
time.sleep(1)

In [2]:
# 색 찾으러 가자...
find_colors = driver.find_element(By.ID, 'facet-drink_color')
color_links = find_colors.find_elements(By.TAG_NAME, 'a')

color_links
links = []
# 링크 하나씩 들어가서 색깔 링크만 뽑기
for color in color_links:
    
    links.append(color.get_attribute('href'))
    
links = links[:-1] # 마지막은 color 아니고 search recipes라 제외
links
# 링크 이용해서 색깔 뽑아내기
colors = []
for link in links:
        color = link.split("/")[3].split("-")[0] # 링크 내부에서 color 뽑기
        colors.append(color)
        
# colors
# print(links)

In [3]:
cocktails_color = {}


color_idx = 0
for url in links:
    driver.get(url)
    pagination_links = driver.find_elements(By.CSS_SELECTOR, 'ul.pagination.js-pager__items li.page-item a')

    # Extract page numbers from pagination links
    page_numbers = [int(link.get_attribute('href').split('=')[-1]) for link in pagination_links]
    if len(page_numbers) == 0:
        max_page = 0
    else:
        max_page = max(page_numbers) + 1
        
    for i in range(max_page):
    # Get the URL of the page
        if max_page != 0:
            page_url = f"{url}/?page={i}"
        else:
            page_url = url
    # Visit the page
        driver.get(page_url)

        # Find all cocktail elements
        cocktail_elements = driver.find_elements(By.CSS_SELECTOR, '.receipe-grid-items')

        # Loop through each cocktail element on the current page
        for cocktail_element in cocktail_elements:
            # Extract name of the cocktail
            name = cocktail_element.find_element(By.CSS_SELECTOR, '.item-desc a').text.strip()

            # Extract link of the cocktail
    #         link = cocktail_element.find_element(By.CSS_SELECTOR, '.item-detail a').get_attribute('href')

            # Add the name and link of the cocktail to the dictionary
            if name not in cocktails_color.keys():
                cocktails_color[name] = []
            cocktails_color[name].append(colors[color_idx])
    color_idx += 1

In [4]:
driver.quit()